# Travelling Santa challenge

Rudolph has always believed in working smarter, not harder. And what better way to earn the respect of Comet and Blitzen than showing the initiative to improve Santa's annual route for delivering toys on Christmas Eve?

This year, Rudolph believes he can motivate the overworked Reindeer team by wisely choosing the order in which they visit the houses on Santa's list. The houses in prime cities always leave carrots for the Reindeers alongside the usual cookies and milk. These carrots are just the sustenance the Reindeers need to keep pace. In fact, Rudolph has found that if the Reindeer team doesn't originate from a prime city exactly every 10th step, it takes them 10% longer than it normally would to make their next destination!

Can you help Rudolph solve the Traveling Santa problem subject to his carrot constraint? His team--and Santa--are counting on you!

<img src="https://mir-s3-cdn-cf.behance.net/project_modules/disp/c8385876438773.5c759e561c2a9.gif" alt="rudolph_compute" width="300"/>

## Prerequisites and Imports
The first step is to import the necessary libraries. These are commonly used ones so the experiment is reproducible.

In [0]:
pip install geopandas

Python interpreter will be restarted.
Collecting geopandas
 Downloading geopandas-0.12.1-py3-none-any.whl (1.1 MB)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from geopandas) (20.9)
Collecting pyproj>=2.6.1.post1
 Downloading pyproj-3.4.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
Collecting fiona>=1.8
 Downloading Fiona-1.8.22-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.6 MB)
Requirement already satisfied: pandas>=1.0.0 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.2.4)
Collecting shapely>=1.7
 Downloading Shapely-1.8.5.post1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.1 MB)
Collecting munch
 Downloading munch-2.5.0-py2.py3-none-any.whl (10 kB)
Requirement already satisfied: setuptools in /usr/local/lib/python3.8/dist-packages (from fiona>=1.8->geopandas) (52.0.0)
Collecting click>=4.0
 Downloading click-8.1.3-py3-none-any.whl (96 kB)
Collecting cligj>=0.5
 Downloading cligj-0.7.2-py3-none-any.whl (7.1 kB)
Requirement already satisfied: six>=1.7 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.15.0)
Requirement already satisfied: attrs>=17 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (20.3.0)
Collecting click-plugins>=1.0
 Downloading click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
Requirement already satisfied: certifi in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2020.12.5)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0.0->geopandas) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0.0->geopandas) (1.19.2)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=1.0.0->geopandas) (2.8.1)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.8/site-packages (from packaging->geopandas) (2.4.7)
Installing collected packages: click, munch, cligj, click-plugins, shapely, pyproj, fiona, geopandas
Successfully installed click-8.1.3 click-plugins-1.1.1 cligj-0.7.2 fiona-1.8.22 geopandas-0.12.1 munch-2.5.0 pyproj-3.4.0 shapely-1.8.5.post1
WARNING: You are using pip version 21.0.1; however, version 22.3.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-cec5fe4c-b67c-4d96-811a-4fa8c9242213/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import numpy as np # library used for working with arrays
import pandas as pd # library used for data manipulation and analysis
from math import radians, cos, sin, asin, sqrt # library used for calculating the distance
import geopandas # library used for spatial operations on geometric types
import plotly.express as px # library used to create visualisations

## Loading the data
The dataset has been extracted from the open source website [simplemaps](https://simplemaps.com/data/world-cities), the top three cities of each country in the world were selected when they had over 200,000 inhabitants.

**Data Dictionary**
* city: name of the city 	
* lat: latitude of the city			
* lng: longitude of the city
* country: name of the city's country
* population: estimate of the city's population
* city_id: generated unique ID
* prime_city: cities for which the city_id is a prime number

In [0]:
# read the data
data = pd.read_csv('../data/worldcities.csv')

### Basic Data Information
This section will provide basic information about the data.

In [0]:
data.head()

Out[3]:

,city,lat,lng,country,population,city_id,prime_city
0,Rovaniemi,66.5028,25.7285,Finland,61763,0,0
1,Tokyo,35.6839,139.7744,Japan,39105000,1,0
2,Jakarta,-6.2146,106.8451,Indonesia,35362000,2,1
3,Delhi,28.6667,77.2167,India,31870000,3,1
4,Manila,14.6000,120.9833,Philippines,23971000,4,0


In [0]:
# check the number of rows and columns in data
data.shape

Out[4]: (369, 7)

In [0]:
# check the data type and if there are any missing values in data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369 entries, 0 to 368
Data columns (total 7 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 city 369 non-null object 
 1 lat 369 non-null float64
 2 lng 369 non-null float64
 3 country 369 non-null object 
 4 population 369 non-null int32 
 5 city_id 369 non-null int32 
 6 prime_city 369 non-null int32 
dtypes: float64(2), int32(3), object(2)
memory usage: 16.0+ KB

In [0]:
# change the prime_city column to a binary data type:
data['prime_city'] = data['prime_city'].astype(bool)

The data frame is composed of 369 rows each representing a city and 7 features. The information above shows that we have no missing values which means we won't have to manipulate the data before running the model. However, the `prime_city` column is shown as an integer but we want to transform it into a binary column, so it is easier to use later on in the script.

Using `mapbox` available in `express` we can place each city from our list on a map to see where Santa is supposed to stop:

In [0]:
# plot the cities in data
fig = px.scatter_mapbox(data, lat="lat", 
                        lon="lng", 
                        hover_name="city", 
                        hover_data=["country","prime_city"],
                        color="city", 
                        zoom=1, 
                        height=500)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

We map above shows how uneven the cities are spread across the globe. We can see dense areas in Europe, Africa, and South America and some more isolated locations in Australia and New Zealand as well as Canada and the United States of America.

## Calculating the Path Distance

In this challenge, we want to optimise the path Santa is going to take to visit all cities on his list. A way to assess the efficiency of a path is to calculate the total distance travelled by Santa and his reindeer. For that, we are going to define a function called `distance()`:
`distance()` takes two arguments:
* `df` a data frame with the city_id, city's coordinates and the prime_city
* `path` a calculated path stored as a list of city IDs

The first step is to define the `previous_stop`. Since we are starting our journey from Santa's home in Lapland every path should start there, this means that the first previous_stop will always be `path[0]`. We then, look at the prime cities and store them as a list oin `prime_cities`. We will be used it later on in the function to increase the distance when the 10th step is not a prime city since we know that it takes the reindeer team 10% longer than it normally would, to make their next destination. We also want to keep track of the `total_distance` starting at zero miles and the `stop_number` starting at stop 1.

We calculate the distance between each city available in the path from the `previous_stop` using the [haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) formula which determines the great-circle distance between two points since Santa and his reindeer can fly using his sleigh and store it in `distance`. We add 10% extra on the mileage every 10th step when it not coming from a prime city and store the final result in `total_distance` until we visited all the cities on our list.

In [0]:
def distance(df,path):
  '''Calculates the distance in miles for a provided path. it takes two arguments:
      - df: a data frame with the city_id, city's coordinates and the prime_city
      - path: a calculated path stored as a list of city IDs
  '''
  previous_stop = path[0] 
  prime_cities = list(df.city_id[df.prime_city == 1]) 
  total_distance = 0
  stop_number = 1 
  for city in path[1:]: 
    next_stop = city 
    # to calculate the total distance of the path, we use haversine distance formula since Santa's can fly using his sleigh 
    # convert decimal degrees to radians 
    lng_previous_stop, lat_previous_stop, lng_next_city, lat_next_city = map(radians, [df.lng[previous_stop], 
                                                                                       df.lat[previous_stop], 
                                                                                       df.lng[next_stop], 
                                                                                       df.lat[next_stop]])
    # haversine formula
    distance_lng = lng_previous_stop - lng_next_city
    distance_lat = lat_previous_stop - lat_next_city

    a = sin(distance_lat/2)**2 + cos(lat_previous_stop) * cos(lat_next_city) * sin(distance_lng/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # radius of earth in miles. Use 6371 for kilometers.
    distance = c * r
    # add 10% extra to the distance when the 10th step is not a prime city:
    if previous_stop in prime_cities:
      extra_distance = distance 
    else:
      extra_distance = distance * (1+ 0.1*((stop_number % 10 == 0))) 
    # add the final distance on the total path distance 
    total_distance = total_distance + extra_distance
    # amend the previous city 
    previous_stop = next_stop
    # add an extra stop the the stop number
    stop_number = stop_number + 1
  return total_distance

## Creating a dumb Path
In this section, we do not try to find the efficient path to take but follow the list order. This will give us a baseline to compare the performance of the different paths.

In [0]:
# calculates the dumb path using the city IDs
dumb_path = list(data.city_id[:].append(pd.Series([0]))) 

In [0]:
# calculating the distance travelled for this path:
print('Total distance with the dumb path is {} miles'.format(round(distance(data,dumb_path),0)))

Total distance with the dumb path is 1783555.0 miles

In [0]:
# merge the dataframe back to the path we created to plot it on a map
df_path = pd.DataFrame({'city_id':dumb_path}).merge(data, how = 'left', on=['city_id'])
df_path.head(5)

Out[11]:

,city_id,city,lat,lng,country,population,prime_city
0,0,Rovaniemi,66.5028,25.7285,Finland,61763,False
1,1,Tokyo,35.6839,139.7744,Japan,39105000,False
2,2,Jakarta,-6.2146,106.8451,Indonesia,35362000,True
3,3,Delhi,28.6667,77.2167,India,31870000,True
4,4,Manila,14.6000,120.9833,Philippines,23971000,False


In [0]:
# plot the path on a map
fig = px.line_mapbox(df_path, 
                     lat="lat", 
                     lon="lng", 
                     hover_name="city", 
                     hover_data=["country","prime_city"],
                     zoom=1, 
                     height=500)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"t":0,"r":0,"b":0,"l":0}) # Dimensions of each margin. (To remember order, think trouble).
fig.show()

# Creating a path ordered by coordinates
In this section, we are trying to improve the path by ordering the coordinates by latitudes and longitudes.

First, we order the cities by coordinates starting with the latitude, excluding the first city since Santa lives in Rovaniemi to add it later at each ends of the list to start and finish at the same location. We can then run our `distance()` function to see if this method improve the mileage.

In [0]:
# order the cities by coordinates
sorted_cities = list(data.iloc[1:,].sort_values(['lat','lng'], ascending=False)['city_id'])
sorted_cities = [0] + sorted_cities + [0]

In [0]:
# calculating the distance travelled for this path
print('Total distance with the sorted cities path is {} miles'.format(round(distance(data,sorted_cities),0)))

Total distance with the sorted cities path is 1099608.0 miles

In [0]:
# merge the path with data
df_path = pd.DataFrame({'city_id':sorted_cities}).merge(data, how = 'left', on=['city_id'])
df_path.head(5)

Out[15]:

,city_id,city,lat,lng,country,population,prime_city
0,0,Rovaniemi,66.5028,25.7285,Finland,61763,False
1,352,Tampere,61.4981,23.7600,Finland,225118,False
2,337,Bergen,60.3925,5.3233,Norway,257087,True
3,331,Espoo,60.2100,24.6600,Finland,269802,True
4,230,Helsinki,60.1756,24.9342,Finland,642045,False


In [0]:
# plot the path on a map
fig = px.line_mapbox(df_path, 
                     lat="lat", 
                     lon="lng", 
                     hover_name="city", 
                     hover_data=["country","prime_city"],
                     zoom=1, 
                     height=500)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

The map above shows Santa going West to East. This is somewhat inefficient. So maybe we could look into ordering the cities but starting with the longitude coordinates:

In [0]:
# order the cities by coordinates
sorted_cities = list(data.iloc[1:,].sort_values(['lng','lat',], ascending=True)['city_id'])
sorted_cities = [0] + sorted_cities + [0]

# calculating the distance travelled for this path:
print('Total distance with the sorted cities path is {} miles'.format(round(distance(data,sorted_cities),0)))

Total distance with the sorted cities path is 505422.0 miles

In [0]:
# merge the path with the data
df_path = pd.DataFrame({'city_id':sorted_cities}).merge(data, how = 'left', on=['city_id'])
df_path.head(5)

Out[18]:

,city_id,city,lat,lng,country,population,prime_city
0,0,Rovaniemi,66.5028,25.7285,Finland,61763,False
1,101,Vancouver,49.2500,-123.1000,Canada,2264823,True
2,26,Los Angeles,34.1139,-118.4068,United States,12750807,False
3,115,Tijuana,32.5250,-117.0333,Mexico,1922523,False
4,47,Guadalajara,20.6767,-103.3475,Mexico,5437000,True


In [0]:
# plot the path on a map
fig = px.line_mapbox(df_path, 
                     lat="lat", 
                     lon="lng", 
                     hover_name="city", 
                     hover_data=["country","prime_city"],
                     zoom=1, 
                     height=500)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

This map shows Santa going North to South. It is more efficient than ordering the cities by latitude but still quite inefficient so we need to find another method to create the path.

# Creating a path using Nearest Neighbours

We want to find the nearest city location to the previous city visited starting from Rovaniemi in Finland which is Santa's home. For that, we will be defining our own function called `nearest_neighbour()`. 
This function finds a path visiting all the cities in Santa's list using nearest neighbour. The principle behind nearest neighbour method is to find the point closest in distance to the new point. In this application, it starts in one city and connects with the closest unvisited one, repeating the same distance calculation until every city has been visited. It then returns to the starting city. For more information on nearest neighbour please visit [this page](https://en.wikipedia.org/wiki/Nearest_neighbour_algorithm).

`nearest_neighbour()` starts by storing multiple information such as a copy of the data called `cities`, it creates an array called `ids` where all the `city_ids` are stored and finally, it stores all the cities' coordinates in an array called `xy`. It then sets the path starting point as `path = [0]` since we started in Rovaniemi.

The next part of the function is a while loop to execute the following statements if the length of the array `ids` is greater than zero, in other words, the loop runs as long as we have cities to visit. This ensures we visit all cities. In the loop, the first thing is to set the coordinates from the previous stop which are stored in the path and therefore equal to `path[-1]`. once we have those set and stored in `last_x, last_y` we can calculate the distances from the previous city to all the remaining cities to visit. The minimum distance is selected, and its index is stored in `nearest_index` and appended to the path. The selected nearest city is then deleted from `ids` and its coordinates from `xy`.

Once all the cities from `ids` have been visited, the loop stops, and we append Rovaniemi as the last city.

In [0]:
# define the function:
def nearest_neighbour():
  '''finds a path through the cities using a nearest neighbour.'''
  cities = data.copy() 
  ids = cities.city_id.values[1:] 
  xy = np.array([cities.lat.values, cities.lng.values]).T[1:] 
  path = [0] 
  while len(ids) > 0: 
    last_x, last_y = cities.lat[path[-1]], cities.lng[path[-1]] 
    dist = ((xy - np.array([last_x, last_y]))**2).sum(-1)
    nearest_index = dist.argmin() 
    path.append(ids[nearest_index]) 
    ids = np.delete(ids, nearest_index, axis=0) 
    xy = np.delete(xy, nearest_index, axis=0)
  path.append(0) 
  return path

nnpath = nearest_neighbour()
print('Total distance with the Nearest Neighbour path is {} miles'.format(round(distance(data,nnpath),0)))

Total distance with the Nearest Neighbour path is 124644.0 miles

In [0]:
# merge the path with the data
df_path = pd.DataFrame({'city_id':nnpath}).merge(data, how = 'left', on=['city_id'])

In [0]:
# plot the path on a map
fig = px.line_mapbox(df_path, 
                     lat="lat", 
                     lon="lng", 
                     hover_name="city", 
                     hover_data=["country","prime_city"],
                     zoom=1, 
                     height=500)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

This path is significantly more efficient than any of the previous ones. However, so far, we did not address the constraint mentioned by Rudolph in this challenge.

# Creating a path using Nearest Neighbours with the constraint
In the challenge, it is mentioned that a penalty of 10% is applied when the 10th visited city is not a prime city location. In this section we will address this problem and see if taking this constraint into account helps reduce the path mileage.

The first step is to have a look how spread the prime cities across the wold map:

In [0]:
# plot the map highliting the prime cities
fig = px.scatter_mapbox(data, 
                        lat="lat", 
                        lon="lng", 
                        hover_name="city",
                        color="prime_city", 
                        zoom=1, height=500)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

We can see that the prime cities are relatively spread across the map.

Next, we will define a function that uses nearest neighbour with the added constraint called `prime_nearest_neighbour()`. This function follows the same steps as the `nearest_neighbour()` with the exception that in the loop, we add if statements:
* In the first if statement we look at if the next step is going to be a prime city, since the penalty applies when we are leaving a prime city, we add 1 to `stop_number`. We also know that the number of prime cities is lesser than the total number of cities so we can only search for a prime city while then we still have some non-visited. Therefore, the next condition is to have a number of non-visited prime cities greater than zero. If both conditions are met, we calculate the distances from the previous stop to all the available prime cities, select the index of the closest one and store it in `nprime_index`. From here we need to find the selected prime location coordinates in the main array `xy`. This is because the indexes in `xy_primes` do not match the indexes in `xy` but `xy` indexes match the data ones. Once we have found the coordinates from the prime location in `xy` we can return the index and store it in `nearest_index` and proceed with removing the city from the various arrays before running the next loop.
If the conditions above are not matching, this means that we do not need to leave from a prime city on the next step and therefore we can proceed as we did in the `nearest_neighbour()`.
* The second if statement is nested in the previous one when the conditions are not met. It checks if the city selected is a prime city if gets located in `xy_primes` array and deleted.
We then add a stop to the total `stop_number` and append the final location to the path.

In [0]:
# define the function
def prime_nearest_neighbour():
  '''finds a path through the cities using a nearest neighbour including the carrot constraint using the prime cities.'''
  cities = data.copy() 
  ids = cities.city_id.values[1:] 
  xy = np.array([cities.lat.values, cities.lng.values]).T[1:] 
  xy_primes = np.array([cities[cities.prime_city == 1].lat.values, cities[cities.prime_city == 1].lng.values]).T[:] 
  path = [0] 
  stop_number = 1 
  while len(ids) > 0: 
    last_x, last_y = cities.lat[path[-1]], cities.lng[path[-1]] 
    if ((stop_number+1) % 10 == 0) and len(xy_primes) > 0:
      dist = ((xy_primes - np.array([last_x, last_y]))**2).sum(-1) 
      nprime_index = dist.argmin()
      nearest_index = np.where(xy == np.array([xy_primes[nprime_index][0], xy_primes[nprime_index][1]]))[0][0] 
      path.append(ids[nearest_index]) 
      ids = np.delete(ids, nearest_index, axis=0) 
      xy = np.delete(xy, nearest_index, axis=0) 
      xy_primes = np.delete(xy_primes, nprime_index, axis=0)
    else:
      dist = ((xy - np.array([last_x, last_y]))**2).sum(-1)
      nearest_index = dist.argmin()
      path.append(ids[nearest_index]) 
      if np.any(xy_primes == np.array([xy[nearest_index][0], xy[nearest_index][1]])): 
        nprime_index = np.where(xy_primes == np.array([xy[nearest_index][0], xy[nearest_index][1]]))[0][0] 
        xy_primes = np.delete(xy_primes, nprime_index, axis=0) 
      ids = np.delete(ids, nearest_index, axis=0) 
      xy = np.delete(xy, nearest_index, axis=0) 
    stop_number = stop_number+1
  path.append(0) 
  return path

pnn_path = prime_nearest_neighbour()
print('Total distance with the Carrot Constraint Nearest Neighbour with path is {} miles'.format(round(distance(data,pnn_path),0)))

Total distance with the Carrot Constraint Nearest Neighbour with path is 158091.0 miles

We notice that the rewards from leaving from a prime city every 10th step does not outperform the penalty of not choosing the next city solely based on its distance. This may be because some prime cities are too far from some world regions and cannot effectively be added without compromising on efficiency.

In [0]:
# merge the path with the data
df_path = pd.DataFrame({'city_id':pnn_path}).merge(data, how = 'left', on=['city_id'])
df_path['stop_number'] = (np.arange(len(df_path)) + 1)

In [0]:
# plot the path on a map
fig = px.line_mapbox(df_path, 
                     lat="lat", 
                     lon="lng", 
                     hover_name="city", 
                     hover_data=["country","prime_city","stop_number"],
                     zoom=1, 
                     height=500)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

We notice that the rewards from leaving from a prime city every 10th step does not outperform the penalty of not choosing the next city solely based on its distance. This may be because some prime cities are too far from some world regions and cannot effectively be added without compromising on efficiency.



Thank you for taking part in this year's workshop! 

Have a lovely Christmas holiday.

<img src="https://mir-s3-cdn-cf.behance.net/project_modules/max_1200/21a4e076438773.5c759e561c5e2.gif" alt="rudolph_thumbs_up" width="300"/>